<a href="https://colab.research.google.com/github/coldDayss/SNS_BigData/blob/main/%EC%A7%80%EB%A7%88%EC%BC%93%EC%97%90%EC%84%9C_Best_Seller_%EC%83%81%ED%92%88_%EC%A0%95%EB%B3%B4_%EC%88%98%EC%A7%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver

import time
import sys
import re
import math
import numpy 
import pandas as pd   
import xlwt 
import random
import os
import urllib.request
import urllib


query_txt='지마켓'
query_url='https://corners.gmarket.co.kr/Bestsellers'


cnt = int(input('    1. 크롤링 할 건수는 몇건입니까?(1-200 건 사이 입력): '))

f_dir = input("    2. 파일을 저장할 폴더명만 쓰세요(예:  C:\Temp\Gmarket  ):")
print("\n")



if cnt > 30 :
      print("    요청 건수가 많아서 시간이 제법 소요되오니 잠시만 기다려 주세요")
else :
      print("    요청하신 데이터를 수집하고 있으니 잠시만 기다려 주세요")
      

now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_dir=f_dir+s+'-'+query_txt
ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'

s_time = time.time( )

from selenium.webdriver.chrome.options import Options

options = Options() 
options.binary_location= 'C:\Program Files\Google\Chrome\Application\chrome.exe'

driver = webdriver.Chrome(chrome_options = options, executable_path=r'C:\Temp\chromedriver.exe')


 
driver.get(query_url)
time.sleep(5)


time.sleep(1)

def scroll_down(driver):
      
      driver.execute_script("window.scrollBy(0,18000);")
      time.sleep(1)

scroll_down(driver)

bmp_map = dict.fromkeys(range(0x10000, sys.maxunicode + 1), 0xfffd)

img_src2=[]   
file_no = 0


html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

reple_result = soup.select('div.best-list')
slist = reple_result[0].find_all('li')

if cnt < 201 :
         
    ranking2=[] #판매순의
    title3=[] #제품 설명
    before_price2=[] #할인 전 가격
    after_price2=[] #할인 후 가격
    score2=[] #할인율
    store2=[]
       
    count = 0
    
   
    img_dir = ff_dir+"\\images"
    os.makedirs(img_dir)
    os.chdir(img_dir)

    for li in slist:
            
            #  이미지를 가져옴
            try :
              photo = li.find('div',class_='thumb').find('img')['src']
            except AttributeError :
              continue
            file_no += 1

            urllib.request.urlretrieve(photo,str(file_no)+'.jpg')
            time.sleep(1)

            if cnt == file_no :
              break
            
            f = open(ff_name, 'a',encoding='UTF-8')
            f.write("-----------------------------------------------------"+"\n")

            # 판매순위 가져옴
            print("-" *70)
            try :
             ranking = li.find('p',class_='no1').get_text()
            except AttributeError :
             ranking = ''
             print(ranking)
            else :
             print("1.판매순위:",ranking)

             f.write('1.판매순위:'+ ranking + "\n")

            #제품 설명 
            try :
             title1 = li.find('a',class_='itemname').get_text().replace("\n","")
            except AttributeError :
             title1 = ''
             print(title1.replace("\n",""))
             f.write('2.제품소개:'+ title1 + "\n")
            else :
             title2=title1.translate(bmp_map).replace("\n","") 
             print("2.제품소개:", title2.replace("\n",""))

             count += 1
             
             f.write('2.제품소개:'+ title2 + "\n")
            
            
            
            # 할인 전 가격
             try :
               before_price = li.find('div','o-price').get_text().replace("\n","")
             except AttributeError :
               before_price = ''
               
             print("3.할인 전 가격:", before_price.replace("\n",""))
             f.write('3.할인 전 가격:'+ before_price + "\n")
                  


            
             # 할인 후 가격
             try :
               after_price = li.find('div','s-price').get_text().replace("\n","")
             except AttributeError :
               after_price = ''
               
             print("4.할인 후 가격:", after_price.replace("\n",""))
             f.write('4.할인 후 가격:'+ after_price + "\n")
                  


            # 할인율 가져오기
             try :
               score = li.find('span','sale').get_text()
             except AttributeError :
               score=' '
               
             print('5.할인율:',score)
             f.write('5.할인율:'+ score + "\n")

             print("-" *70)
                          
             f.close( )
              
             time.sleep(0.3)
            
             ranking2.append(ranking)
             title3.append(title2.replace("\n",""))
             before_price2.append(before_price.replace("\n",""))
             after_price2.append(after_price.replace("\n",""))
             score2.append(score)

             if count == cnt :
                break
                          


else :
      print(" 검색 건수는 1건 - 최대 200 건까지만 가능합니다")
  
              
Gmarket_best_seller = pd.DataFrame()
Gmarket_best_seller['판매순위']=ranking2
Gmarket_best_seller['제품소개']=pd.Series(title3)
Gmarket_best_seller['할인 전 판매가격']=pd.Series(before_price2)
Gmarket_best_seller['할인 후 판매가격']=pd.Series(after_price2)
Gmarket_best_seller['상품 할인율']=pd.Series(score2)


# csv 형태로 저장하기
Gmarket_best_seller.to_csv(fc_name,encoding="utf-8-sig",index=True)

# 엑셀 형태로 저장하기
Gmarket_best_seller.to_excel(fx_name ,index=True)

e_time = time.time( )
t_time = e_time - s_time

# txt 파일에 크롤링 요약 정보 저장하기
orig_stdout = sys.stdout
f = open(ff_name, 'a',encoding='UTF-8')
sys.stdout = f


#이미지 삽입하기

import win32com.client as win32 
import win32api 
                    
excel = win32.gencache.EnsureDispatch('Excel.Application')
wb = excel.Workbooks.Open(fx_name)
sheet = wb.ActiveSheet
sheet.Columns(3).ColumnWidth = 30  
row_cnt = cnt+1
sheet.Rows("2:%s" %row_cnt).RowHeight = 120  
ws = wb.Sheets("Sheet1")
col_name2=[]
file_name2=[]

for a in range(2,cnt+2) :
      col_name='C'+str(a)
      col_name2.append(col_name)

for b in range(1,cnt+1) :
      file_name=img_dir+'\\'+str(b)+'.jpg'
      file_name2.append(file_name)
      
for i in range(0,cnt) :
      rng = ws.Range(col_name2[i])
      image = ws.Shapes.AddPicture(file_name2[i], False, True, rng.Left, rng.Top, 130, 100)
      excel.Visible=True
      excel.ActiveWorkbook.Save()


print("\n")
print("=" *50)
print("총 소요시간은 %s 초 이며," %t_time)
print("총 저장 건수는 %s 건 입니다 " %count)
print("=" *50)

sys.stdout = orig_stdout
f.close( )

print("\n") 
print("=" *50)
print("1.요청된 총 %s 건의 리뷰 중에서 실제 크롤링 된 리뷰수는 %s 건입니다" %(cnt,count))
print("2.총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("3.파일 저장 완료: txt 파일명 : %s " %ff_name)
print("4.파일 저장 완료: csv 파일명 : %s " %fc_name)
print("5.파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *50)    

    1. 크롤링 할 건수는 몇건입니까?(1-00 건 사이 입력): 10
    2. 파일을 저장할 폴더명만 쓰세요(예:  C:\Temp\Gmarket  ):C:\Temp\Gmarket


<ipython-input-5-c60c5a01d775>:52: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options = options, executable_path=r'C:\Temp\chromedriver.exe')


com_error: (-2147352567, '예외가 발생했습니다.', (0, None, '지정한 파일이 없습니다.', None, 0, -2146827284), None)